In [2]:
import pandas as pd
import numpy as np

In [3]:
def read_data_frame(name):
    return pd.DataFrame.from_csv('../../datasets/recordings/' + name + '.csv')

df = read_data_frame('NSH2_iPhone')
df_labels = read_data_frame('NSH2_iPhone_labels')

In [3]:
# training and testing split
indexes = df_labels.index.tolist()
np.random.shuffle(indexes)
split = round(0.7*len(indexes))
training_i = indexes[:split]
testing_i = indexes[split:]
df_train = df.loc[df.id.isin(training_i)]
df_test = df.loc[df.id.isin(testing_i)]

In [4]:
y_train = df_labels.loc[df_labels.index.isin(training_i)]
y_test = df_labels.loc[df_labels.index.isin(testing_i)]

y_train = y_train['label']
y_test = y_test['label']

X_train = pd.DataFrame(index=y_train.index)
X_test = pd.DataFrame(index=y_test.index)

In [46]:
from tsfresh.transformers import RelevantFeatureAugmenter, FeatureAugmenter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.decomposition import PCA

In [47]:
ppl = Pipeline([
    ('fresh', RelevantFeatureAugmenter(column_id='id')),
    ('impute', Imputer()),
    ('scale', StandardScaler()),
    ('clf', RandomForestClassifier())
])

ppl.set_params(fresh__timeseries_container=df_train)
ppl.fit(X_train, y_train)

Feature Extraction: 100%|██████████| 34/34 [00:11<00:00,  2.96it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature 31__sum_of_reoccurring_values is constant



Pipeline(steps=[('fresh', RelevantFeatureAugmenter(column_id=None, column_kind=None, column_sort=None,
             column_value=None, evaluate_only_added_features=True,
             feature_extraction_settings=None,
             feature_selection_settings=None,
             timeseries_container=            ...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

In [48]:
ppl.set_params(fresh__timeseries_container=df_test)

y_pred = ppl.predict(X_test)

print(classification_report(y_test, y_pred))

Feature Extraction: 100%|██████████| 34/34 [00:05<00:00,  6.25it/s]WARNING:tsfresh.feature_selection.significance_tests:[target_binary_feature_binary_test] The binary target should have values 1 and 0 (or True and False). Instead found{1.0, 3.0}


             precision    recall  f1-score   support

          1       1.00      1.00      1.00        12
          3       1.00      1.00      1.00        10

avg / total       1.00      1.00      1.00        22

